In [24]:
%matplotlib inline
%time from hikyuu.interactive.interactive import *
use_draw_engine('echarts')

Wall time: 6.63 s


# 一、策略分析

## 原始描述

建仓条件：expma周线exp1跟exp2金叉向上使用使用 B=50% 的资金买入股票，建仓成功后，卖出条件才能起作用

卖出条件S1：expma日线exp1和exp2死叉向下时卖出持仓股 S=50%

买入条件B1：expma日线exp1和exp2金叉向上时买入股票数为S（卖出条件S1卖出股数）

S1和B1就这样循环

清仓条件为：expma周线exp1和exp2死叉时


## 策略分析

市场环境：无

系统有效性：周线EMA1（快线）和EMA2（慢线）金叉向上直到两者死叉，系统有效时建立初始仓位

信号指示器：
- 买入：日线EMA1（快线）和EMA2（慢线）金叉向上
- 卖出：日线EMA1（快线）和EMA2（慢线）死叉向下

止损/止盈：无

资金管理：
- 初次建仓：使用50%的资金
- 买入：初次建仓时持股数的50%
- 卖出：初次建仓时持股数的50%

盈利目标：无


# 二、实现系统部件

## 自定义系统有效性策略

In [67]:
def getNextWeekDateList(week):
    from datetime import timedelta
    py_week = week.datetime()
    next_week_start = py_week + timedelta(days = 7 - py_week.weekday())
    next_week_end = next_week_start + timedelta(days=5)
    return getDateRange(Datetime(next_week_start), Datetime(next_week_end))
#ds = getNextWeekDateList(Datetime(201801010000))
#for d in ds:
#    print(d)

In [59]:
def DEMO_CN(self):
    """ DIF > DEA 时，系统有效
    参数：
    fast_n：周线dif窗口
    slow_n: 周线dea窗口
    """
    k = self.getTO()
    if (len(k) <= 1):
        return
    
    #-----------------------------
    # 周线        
    #-----------------------------
    week_q = QueryByDate(k[0].datetime, k[-1].datetime, kType=Query.WEEK)
    week_k = k.getStock().getKData(week_q)
    
    n1 = self.getParam("week_macd_n1")
    n2 = self.getParam("week_macd_n2")
    n3 = self.getParam("week_macd_n3")
    m = MACD(CLOSE(week_k), n1, n2, n3)
    fast = m.getResult(0)
    slow = m.getResult(1)

    x = fast > slow
    for i in range(x.discard, len(x)-1):
        if (x[i] >= 1.0):
            #需要被扩展到日线（必须是后一周）
            date_list = getNextWeekDateList(week_k[i].datetime)
            for d in date_list:
                self._addValid(d)

## 自定义信号指示器

In [60]:
#不需要，已经有内建的SG_Cross函数可直接使用

## 自定义资金管理策略

In [61]:
class DEMO_MM(MoneyManagerBase):
    """
    初次建仓：使用50%的资金
    买入：初次建仓时持股数的50%
    卖出：初次建仓时持股数的50%
    """
    def __init__(self):
        super(DEMO_MM, self).__init__("MACD_MM")
        
    def _reset(self):
        pass
        
    def _clone(self):
        mm = DEMO_MM()
        return mm
    
    def _getBuyNumber(self, datetime, stk, price, risk):
        tm = self.getTM()
        cash = tm.currentCash
        
        position = tm.getPosition(stk)
        current_num = position.number
        
        if current_num == 0:
            #仓位为0，为初次建仓
            num = int((cash * 0.5 // price // stk.atom) * stk.atom)
        else:
            #非初次建仓，买入同等数量
            num = current_num
        
        return num
    
    def _getSellNumber(self, datetime, stk, price, risk):
        tm = self.getTM()
        position = tm.getPosition(stk)
        current_num = position.number
        return int(current_num*0.5)

# 三、构建并运行系统

## 设定系统参数

In [62]:
#账户参数
init_cash = 100000 #账户初始资金
init_date = '1990-1-1' #账户建立日期

#信号指示器参数
week_n1 = 12
week_n2 = 26
week_n3 = 9


## 构建系统实例

In [63]:
#创建模拟交易账户进行回测，初始资金30万
my_tm = crtTM(datetime=Datetime(init_date), initCash = init_cash)

#创建系统实例
my_sys = SYS_Simple()

my_sys.tm = my_tm
my_sys.cn = crtCN(DEMO_CN, 
              {'week_macd_n1': week_n1, 'week_macd_n2': week_n2, 'week_macd_n3': week_n3}, 
                'DEMO_CN')  
my_sys.sg = SG_Cross(OP(EMA(n=week_n1)), OP(EMA(n=week_n2)))
my_sys.mm = DEMO_MM()

## 运行系统

In [64]:
#待测试对象及数据选择
stk = sm['sz000001']
q = Query(-300, kType=Query.DAY) 

my_sys.setParam("cn_open_position", False)
my_sys.run(stk, q)

my_tm.tocsv(sm.tmpdir())

# 四、查看资金曲线及绩效统计

In [57]:
#绘制资金收益曲线
x = my_tm.getProfitCurve(stk.getDatetimeList(q), KQuery.DAY)
#x = my_tm.getFundsCurve(stk.getDatetimeList(q), KQuery.DAY) #资金净值曲线
x = PRICELIST(x)
x.plot()

In [69]:
#回测统计
per = Performance()
print(per.report(my_tm, Datetime.now()))

帐户初始金额: 100000.00
累计投入本金: 100000.00
累计投入资产: 0.00
累计借入现金: 0.00
累计借入资产: 0.00
累计红利: 0.00
现金余额: 110746.00
未平仓头寸净值: 0.00
当前总资产: 110746.00
已平仓交易总成本: 0.00
已平仓净利润总额: 10746.00
单笔交易最大占用现金比例%: 49.90
交易平均占用现金比例%: 43.98
已平仓帐户收益率%: 10.75
帐户年复合收益率%: 9.42
帐户平均年收益率%: 9.47
赢利交易赢利总额: 10746.00
亏损交易亏损总额: 0.00
已平仓交易总数: 2.00
赢利交易数: 2.00
亏损交易数: 0.00
赢利交易比例%: 100.00
赢利期望值: 5373.00
赢利交易平均赢利: 5373.00
亏损交易平均亏损: 0.00
平均赢利/平均亏损比例: 0.00
净赢利/亏损比例: 0.00
最大单笔赢利: 8856.00
最大单笔亏损: 0.00
赢利交易平均持仓时间: 89.50
赢利交易最大持仓时间: 172.00
亏损交易平均持仓时间: 0.00
亏损交易最大持仓时间: 0.00
空仓总时间: 236.00
空仓时间/总时间%: 56.00
平均空仓时间: 118.00
最长空仓时间: 169.00
最大连续赢利笔数: 2.00
最大连续亏损笔数: 0.00
最大连续赢利金额: 10746.00
最大连续亏损金额: 0.00
R乘数期望值: 0.08
交易机会频率/年: 1.76
年度期望R乘数: 0.14
赢利交易平均R乘数: 0.08
亏损交易平均R乘数: 0.00
最大单笔赢利R乘数: 0.12
最大单笔亏损R乘数: 0.00
最大连续赢利R乘数: 0.08
最大连续亏损R乘数: 0.00



# 五、或许想看下图形

In [66]:
ax1, ax2 = create_figure(2)
my_sys.plot(axes=ax1)
m = MACD()

# 六、或许想看看所有股票的情况

In [ ]:
import pandas as pd
def calTotal(blk, q):
    per = Performance()
    s_name = []
    s_code = []
    x = []
    for stk in blk:
        my_sys.run(stk, q)
        per.statistics(my_tm, Datetime.now())
        s_name.append(stk.name)
        s_code.append(stk.market_code)
        x.append(per.get("当前总资产".encode('gb2312')))
    return pd.DataFrame({'代码': s_code, '股票': s_name, '当前总资产': x})

%time data = calTotal(blocka, q)

In [ ]:
#保存到CSV文件
#data.to_csv(sm.tmpdir() + '/统计.csv')
data[:10]